In [4]:
import requests
import json
import pandas as pd
from datetime import datetime
today_date = datetime.now().strftime('%Y_%m_%d')

url = "https://frapi.marketsmojo.com/screener/search"

pagenum = [1, 2]
market = []
for i in pagenum:
    payload = {"screener_type":1, "pagenum":i,"filters":"[{'marketcap':[1,2]},{'result_quarter':[1]},{'decleared':[[{'field':'decleared','Display':'RESULTS DECLARED','pos':6,'stop':true,'checked':true,'list':[]}]]}]","sort":"[{'col':'fin_score','dir':-1}]","resultcorner":"1","resulttype":"latest"}
    res = requests.post(url, json=payload)
    restext = '[' + res.text + ']'
    json_text = json.loads(restext)
    for i in range(0, len(json_text[0]['data']['list'])):
        mcap_grade = json_text[0]['data']['list'][i]['mcap_grade'].strip().lower().replace(' ','')
        if mcap_grade == 'largecap' or mcap_grade == 'midcap':
            company  = json_text[0]['data']['list'][i]['sname']
            price = json_text[0]['data']['list'][i]['price']
            market_cap =str(json_text[0]['data']['list'][i]['mcap']).replace('Cr)','').replace('(', '').replace(',','').strip()
            industry = json_text[0]['data']['list'][i]['ind_name']
            financial_trend = json_text[0]['data']['list'][i]['f_txt']+' '+json_text[0]['data']['list'][i]['f_score']
            Result_commentary = json_text[0]['data']['list'][i]['fin_commentory']+' '+json_text[0]['data']['list'][i]['fin_score_t']
            Result_date = json_text[0]['data']['list'][i]['result']
            returns_since_result = json_text[0]['data']['list'][i]['resultdt']
            returns_last_15_days = json_text[0]['data']['list'][i]['result15']
            sentiment = json_text[0]['data']['list'][i]['f_txt']
            
            a=[company,price,market_cap,industry,financial_trend,Result_commentary,Result_date,returns_since_result,returns_last_15_days,sentiment]
            market.append(a)

df = pd.DataFrame(market, columns=['company','price','market_cap','industry','financial_trend','Result_commentary','Result_date','returns_since_result','returns_last_15_days','sentiment'])
df = df.drop_duplicates()
count=df['sentiment'].value_counts()
df['Flat percent'] = count[0]/len(df)*100
df['Negative percent'] = count[1]/len(df)*100
df['Positive percent'] = count[2]/len(df)*100
df['scraped_date']=today_date

df.to_csv('MojoMarket_data.csv', index=False)

